<a href="https://colab.research.google.com/github/Sahilchawla1094/Travel-package-prediction/blob/main/Travel_package_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
#To install xgboost library use - 
#!pip install xgboost
### IMPORT: ------------------------------------
import scipy.stats as stats 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

import statsmodels.api as sm
#--Sklearn library--
from sklearn.model_selection import train_test_split # Sklearn package's randomized data splitting function

from sklearn import metrics

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression

# Libtune to tune model, get different metric scores

from sklearn.metrics import  classification_report, accuracy_score, precision_score, recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,plot_confusion_matrix #to plot confusion matric

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth',400)
# To supress numerical display in scientific notations
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
warnings.filterwarnings('ignore') # To supress warnings
 # set the background for the graphs
plt.style.use('ggplot')
# For pandas profiling
from pandas_profiling import ProfileReport
print('Load Libraries-Done')
#!pip install xlrd
#!!pip install openpyxl


Load Libraries-Done
Load Libraries-Done


In [ ]:
df = pd.read_excel('/content/gdrive/My Drive/capstone/Tourism.xlsx')

In [ ]:
df_tour=df.copy()
print(f'There are {df_tour.shape[0]} rows and {df_tour.shape[1]} columns') # fstring 

There are 4888 rows and 20 columns


In [ ]:
df_tour.head()

,CustomerID,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
0,200000.00000,1.00000,41.00000,Self Enquiry,3.00000,6.00000,Salaried,Female,3.00000,3.00000,Super Deluxe,3.00000,Single,1.00000,1.00000,2.00000,1.00000,0.00000,Manager,20993.00000
1,200001.00000,0.00000,49.00000,Company Invited,1.00000,14.00000,Salaried,Male,3.00000,4.00000,Super Deluxe,4.00000,Divorced,2.00000,0.00000,3.00000,1.00000,2.00000,Manager,20130.00000
2,200002.00000,1.00000,37.00000,Self Enquiry,1.00000,8.00000,Free Lancer,Male,3.00000,4.00000,Multi,3.00000,Single,7.00000,1.00000,3.00000,0.00000,0.00000,Executive,17090.00000
3,200003.00000,0.00000,33.00000,Company Invited,1.00000,9.00000,Salaried,Female,2.00000,3.00000,Multi,3.00000,Divorced,2.00000,1.00000,5.00000,1.00000,1.00000,Executive,17909.00000
4,200004.00000,0.00000,NaN,Self Enquiry,1.00000,8.00000,Small Business,Male,2.00000,3.00000,Multi,4.00000,Divorced,1.00000,0.00000,5.00000,1.00000,0.00000,Executive,18468.00000


In [ ]:
df_tour.tail()

,CustomerID,ProdTaken,Age,PreferredLoginDevice,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
4883,204883.00000,1.00000,49.00000,Self Enquiry,3.00000,9.00000,Small Business,Male,3.00000,5.00000,Super Deluxe,4.00000,Unmarried,2.00000,1.00000,1.00000,1.00000,1.00000,Manager,26576.00000
4884,204884.00000,1.00000,28.00000,Company Invited,1.00000,31.00000,Salaried,Male,4.00000,5.00000,Multi,3.00000,Single,3.00000,1.00000,3.00000,1.00000,2.00000,Executive,21212.00000
4885,204885.00000,1.00000,52.00000,Self Enquiry,3.00000,17.00000,Salaried,Female,4.00000,4.00000,Standard,4.00000,Married,7.00000,0.00000,1.00000,1.00000,3.00000,Senior Manager,31820.00000
4886,204886.00000,1.00000,19.00000,Self Enquiry,3.00000,16.00000,Small Business,Male,3.00000,4.00000,Multi,3.00000,Single,3.00000,0.00000,5.00000,0.00000,2.00000,Executive,20289.00000
4887,204887.00000,1.00000,36.00000,Self Enquiry,1.00000,14.00000,Salaried,Male,4.00000,4.00000,Multi,4.00000,Unmarried,3.00000,1.00000,3.00000,1.00000,2.00000,Executive,24041.00000


**Understand the dataset**

In [ ]:
#get the size of dataframe
print ("Rows     : " , df_tour.shape[0])  #get number of rows/observations
print ("Columns  : " , df_tour.shape[1]) #get number of columns
print ("#"*40,"\n","Features : \n\n", df_tour.columns.tolist()) #get name of columns/features
print ("#"*40,"\nMissing values :\n\n", df_tour.isnull().sum().sort_values(ascending=False))
print( "#"*40,"\nPercent of missing :\n\n", round(df_tour.isna().sum() / df_tour.isna().count() * 100, 2).sort_values(ascending=False)) # looking at columns with most Missing Values

Rows     :  4888
Columns  :  20
######################################## 
 Features : 

 ['CustomerID', 'ProdTaken', 'Age', 'PreferredLoginDevice', 'CityTier', 'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisited', 'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar', 'MaritalStatus', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'NumberOfChildrenVisited', 'Designation', 'MonthlyIncome']
######################################## 
Missing values :

 DurationOfPitch            251
MonthlyIncome              233
Age                        226
NumberOfTrips              140
NumberOfChildrenVisited     66
NumberOfFollowups           45
PreferredPropertyStar       26
PreferredLoginDevice        25
Designation                  0
OwnCar                       0
PitchSatisfactionScore       0
Passport                     0
CustomerID                   0
MaritalStatus                0
ProdTaken                    0
NumberOfPersonVisited        0
Gender    

In [ ]:
df_tour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CustomerID               4888 non-null   float64
 1   ProdTaken                4888 non-null   float64
 2   Age                      4662 non-null   float64
 3   PreferredLoginDevice     4863 non-null   object 
 4   CityTier                 4888 non-null   float64
 5   DurationOfPitch          4637 non-null   float64
 6   Occupation               4888 non-null   object 
 7   Gender                   4888 non-null   object 
 8   NumberOfPersonVisited    4888 non-null   float64
 9   NumberOfFollowups        4843 non-null   float64
 10  ProductPitched           4888 non-null   object 
 11  PreferredPropertyStar    4862 non-null   float64
 12  MaritalStatus            4888 non-null   object 
 13  NumberOfTrips            4748 non-null   float64
 14  Passport                

**Observations**

There are lot of missing values.

5% values are missing in DurationOfPitch, that may be cause customer was never pitched, need to analyze further

4.77% values are missig from MonthlyIncome.

Age has 4.62 % missing values.

NumberOfChildrenVisited,NumberOfFollowups,NumberOfTrips,PreferredPropertyStar,
PreferredLoginDevice has less than 3% missing values

ProdTaken is the Target Variable

Designation,MartialStatus,ProductPitched,Gender,OwnCar,Passport,CityTier,
PreferredPropertyStar, Occupation,PreferredLoginDevice are categorical variables 
while others are numerical value.

**Summary of the dataset**

In [ ]:
df_tour.describe().T